In [ ]:
def pdf_to_img(pdf_file):
    return(pdf2image.convert_from_path(pdf_file))

In [ ]:
def ocr_core(file):
    text = pytesseract.image_to_string(file)
    return(text)

In [ ]:
def print_pages(pdf_file):
    images = pdf_to_img(pdf_file)
    for pg, img in enumerate(images):
        return(ocr_core(img))

In [ ]:
def new_id(file, tipo):
    DF_BBDD = pd.read_excel(file)
    if(DF_BBDD.shape[0]>0):
        lastid = DF_BBDD['ID'].tail(1).tolist()[0].split('-')
        new_id = lastid[0]+'-'+str(int(lastid[1].lstrip('0'))+1).zfill(9)
    elif(DF_BBDD.shape[0]==0):
        if(tipo == 'CLIENTE'):
            new_id = 'C-000000001'
        elif(tipo == 'PROVEEDOR'):
            new_id = 'P-000000001'
    return(new_id)

In [ ]:
def treat_text(lista):
    lista = [x.lower() for x in lista]
    lista = [re.sub(r'[^\w]', ' ', x) for x in lista]
    return([unidecode.unidecode(x) for x in lista])

In [ ]:
def get_info_prov(df_proveedores, factura):
    
    if(df_proveedores.shape[0]>0):
        
        ref_name = df_proveedores['CIF'].tolist()
        ref_names_t = treat_text(ref_name)
        factura_text_t = treat_text(factura)

        match=[]
        for i in range(len(ref_names_t)):
            for j in range(len(factura_text_t)):
                if(ref_names_t[i] in factura_text_t[j]):
                    match.append(str(i))
        if(len(match)>0):
            output = df_proveedores.iloc[int(match[0]),:]
            return(output)
        else:
            print('Proveedor no registrado. Por favor, cree un nuevo proveedor de servicios.')
            %run P01_CREAR_PROVEEDOR.ipynb
            
    else:
        print('Base de datos no creada, Por favor, cree un nuevo proveedor de servicios.')
        %run P01_CREAR_PROVEEDOR.ipynb

In [ ]:
def get_info_cli(df_clientes, factura):

    if(df_clientes.shape[0]>0):
        
        ref_name = df_clientes['CIF'].tolist()
        ref_names_t = treat_text(ref_name)
        factura_text_t = treat_text(factura)

        match=[]
        for i in range(len(ref_names_t)):
            for j in range(len(factura_text_t)):
                if(ref_names_t[i] in factura_text_t[j]):
                    match.append(str(i))
        if(len(match)>0):
            output = df_clientes.iloc[int(match[0]),:]
            return(output)
        else:
            print('Cliente no registrado. Por favor, cree un nuevo cliente.')
            %run P01_CREAR_CLIENTE.ipynb
            
    else:
        print('Base de datos no creada, Por favor, cree un nuevo cliente.')
        %run P01_CREAR_CLIENTE.ipynb

In [ ]:
def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

In [ ]:
def get_fecha(inputs):
    meses = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']
    FEM = [inputs[i] for i in [i for i in range(len(inputs)) if any(mes in inputs[i] for mes in meses)]]
    if(len(FEM) == 0):
        candidatas = [re.search('([0-9|/]+)|([0-9|-]+)',line).group() for line in inputs if re.search('([0-9|/]+)|([0-9|-]+)',line) != None]
        fecha =[x for x in candidatas if re.search('([1-9]|1[0-9]|2[0-9]|3[0-1]|0[0-9])(.|-|\/)([1-9]|1[0-2]|0[0-9])(.|-|\/)(20[0-9][0-9])|([1-9]|1[0-2]|0[0-9])(.|-|\/)([1-9]|1[0-9]|2[0-9]|3[0-1]|0[0-9])(.|-|\/)(20[0-9][0-9])',x)][0]
    else:
        fecha = FEM[0]
    return(fecha)

In [ ]:
def format_info(tipo, ruta_fact):
    
    factura_raw = print_pages(ruta_fact)
    df_fact_treat = factura_raw.split('\n')
    
    if (tipo == 'COMPRA'):
        DF_BBDD = pd.read_excel('../P01_BBDD/DB_PROVEEDORES.xlsx')
        info_fact = pd.DataFrame(get_info_prov(DF_BBDD, df_fact_treat)).transpose()
    elif(tipo == 'VENTA'):
        DF_BBDD = pd.read_excel('../P01_BBDD/DB_CLIENTES.xlsx')
        info_fact = pd.DataFrame(get_info_cli(DF_BBDD, df_fact_treat)).transpose()
         
    tables = tabula.read_pdf(ruta_fact, pages = "all", multiple_tables = True)
    df_dig = pd.DataFrame()
    for table in tables:
        if len(table) > 0:
            df_dig = table
    datos = pd.DataFrame(np.vstack([df_dig.columns, df_dig])).rename(columns={0:'CONCEPTO',1:'VALOR'})

    df_info = info_fact[['ID', 'NOMBRE', 'CIF', 'CP']].transpose().reset_index().rename(columns={'index':'CONCEPTO',0:'VALOR'})
    fecha = get_fecha(df_fact_treat)
    fecha = pd.DataFrame(['FECHA', fecha]).transpose().rename(columns={0:'CONCEPTO',1:'VALOR'})
    fecha_conta = str(date.today())
    fecha_conta = pd.DataFrame(['FECHA_CONTA', fecha_conta]).transpose().rename(columns={0:'CONCEPTO',1:'VALOR'})
    output = pd.concat([df_info, fecha, fecha_conta, datos], axis=0).reset_index(drop=True)
    
    return(output)

In [ ]:
def excel(): 
      
    # resize the width of columns in 
    # excel spreadsheet 
    sheet.column_dimensions['A'].width = 50
    sheet.column_dimensions['B'].width = 50
    sheet.column_dimensions['C'].width = 50
    sheet.column_dimensions['D'].width = 50
    sheet.column_dimensions['E'].width = 50
    sheet.column_dimensions['F'].width = 50
    sheet.column_dimensions['G'].width = 50
    sheet.column_dimensions['H'].width = 50
    sheet.column_dimensions['I'].width = 50
    sheet.column_dimensions['J'].width = 50
    sheet.column_dimensions['K'].width = 50
    sheet.column_dimensions['L'].width = 55
    sheet.column_dimensions['M'].width = 50
    sheet.column_dimensions['N'].width = 50
    sheet.column_dimensions['O'].width = 50
  
    # write given data to an excel spreadsheet 
    # at particular location 
    
    sheet.cell(row=1, column=1).value = "ID"
    sheet.cell(row=1, column=2).value = "NOMBRE"
    sheet.cell(row=1, column=3).value = "RAZON"
    sheet.cell(row=1, column=4).value = "CIF"
    sheet.cell(row=1, column=5).value = "DIRECCION"
    sheet.cell(row=1, column=6).value = "CP"
    sheet.cell(row=1, column=7).value = "LOCALIDAD"
    sheet.cell(row=1, column=8).value = "PROVINCIA"
    sheet.cell(row=1, column=9).value = "CCAA"
    sheet.cell(row=1, column=10).value = "PAIS"
    sheet.cell(row=1, column=11).value = "TEL"
    sheet.cell(row=1, column=12).value = "MOVIL"
    sheet.cell(row=1, column=13).value = "FAX"
    sheet.cell(row=1, column=14).value = "EMAIL"
    sheet.cell(row=1, column=15).value = "WEB"

In [ ]:
# Function to set focus (cursor) 
def focus1(event): 
    # set focus on the course_field box 
    return(new_id(file, tipo))
  
  
# Function to set focus 
def focus2(event): 
    # set focus on the sem_field box 
    nombre_field.focus_set() 
  
  
# Function to set focus 
def focus3(event): 
    # set focus on the form_no_field box 
    razon_field.focus_set() 

# Function to set focus 
def focus4(event): 
    # set focus on the form_no_field box 
    cif_field.focus_set()
  
# Function to set focus 
def focus5(event): 
    # set focus on the contact_no_field box 
    direccion_field.focus_set() 
  
  
# Function to set focus 
def focus6(event): 
    # set focus on the email_id_field box 
    cp_field.focus_set() 
  
  
# Function to set focus 
def focus7(event): 
    # set focus on the address_field box 
    loc_field.focus_set() 
    
# Function to set focus 
def focus8(event): 
    # set focus on the address_field box 
    loc_field.focus_set() 

# Function to set focus 
def focus9(event): 
    # set focus on the address_field box 
    loc_field.focus_set() 
    
# Function to set focus 
def focus10(event): 
    # set focus on the address_field box 
    loc_field.focus_set() 
    
# Function to set focus 
def focus11(event): 
    # set focus on the address_field box 
    loc_field.focus_set() 
    
# Function to set focus 
def focus12(event): 
    # set focus on the address_field box 
    loc_field.focus_set() 
    
# Function to set focus 
def focus13(event): 
    # set focus on the address_field box 
    loc_field.focus_set() 
    
# Function to set focus 
def focus14(event): 
    # set focus on the address_field box 
    loc_field.focus_set() 
    
# Function to set focus 
def focus15(event): 
    # set focus on the address_field box 
    loc_field.focus_set() 

In [ ]:
# Function for clearing the 
# contents of text entry boxes 
def clear(): 
      
    # clear the content of text entry box 
    id_field.delete(0, END) 
    nombre_field.delete(0, END) 
    razon_field.delete(0, END)
    cif_field.delete(0, END)
    direccion_field.delete(0, END) 
    cp_field.delete(0, END) 
    loc_field.delete(0, END) 
    prov_field.delete(0, END)
    ccaa_field.delete(0, END) 
    pais_field.delete(0, END) 
    tel_field.delete(0, END) 
    mov_field.delete(0, END) 
    fax_field.delete(0, END) 
    email_field.delete(0, END) 
    web_field.delete(0, END) 

In [ ]:
def close_window():
    root.destroy()

In [ ]:
def new_proveedor():
    %run P01_CREAR_PROVEEDOR.ipynb

In [ ]:
def new_cliente():
    %run P01_CREAR_CLIENTE.ipynb

In [ ]:
def open_BBDD_proveedor():
    
    root_prov = Tk()
    root_prov.title("BBDD PROVEEDORES") 
    root_prov.geometry("800x400") 
    
    frame = Frame(root_prov)
    frame.pack(fill='both', expand=True)
    tabla_prov = pd.read_excel('../P01_BBDD/DB_PROVEEDORES.xlsx')
    pt_prov = Table(frame, dataframe=tabla_prov)
    pt_prov.show()
    
    root_prov.mainloop()

In [ ]:
def open_BBDD_cliente():
    
    root_cli = Tk()
    root_cli.title("BBDD CLIENTES") 
    root_cli.geometry("800x400") 
    
    frame = Frame(root_cli)
    frame.pack(fill='both', expand=True)
    tabla_cli = pd.read_excel('../P01_BBDD/DB_CLIENTES.xlsx')
    
    pt_cli = Table(frame, dataframe=tabla_cli)
    pt_cli.show()
    
    root_cli.mainloop()

In [ ]:
def print_table(tabla_conta):
    
    root_tab = Tk()
    root_tab.title("Tabla") 
    root_tab.geometry("800x400") 
    
    frame = Frame(root_tab)
    frame.pack(fill='both', expand=True)
    
    pt_tab = Table(frame, dataframe=tabla_conta)
    pt_tab.show()
    
    root_tab.mainloop()